In [9]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

from sklearn.preprocessing import QuantileTransformer
from joblib import load

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model.joblib")


In [10]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [11]:
df = pd.read_sql(
    """
    select ticker, roeq, bm, mom12m, mve, famaindustry
    from today
    """, 
    conn
)
df = df.dropna()
df = df.set_index("ticker")

In [12]:
df = df[df.price>=5]
df = df.sort_values(by="mve")
df = df.iloc[:-500]

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kerry\AppData\Local\Temp\ipykernel_18348\2950429033.py", line 1, in <module>
    df = df[df.price>=5]
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py", line 5907, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'price'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\pool\base.py", line 753, in _finalize_fairy
    fairy._reset(pool)
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\pool\base.py", line 1004

AttributeError: 'DataFrame' object has no attribute 'price'

In [ ]:
features = ["roeq", "bm", "mom12m"]
df[features] = qt.fit_transform (df[features])
df["predict"] = pipe.predict(df[features])
df.to_csv("files/predictions.csv")